In [ ]:
import app.agent as agent

In [ ]:
graph = agent.build_agent_with_router()

In [ ]:
user_text = "Удали таблицу fact_sales из базы данных"

msg = {"messages": [{"role": "user", "content": user_text}]}
config = {"configurable": {"thread_id": str(1)}}
print(config)
response = graph.invoke(msg, config)
last_msg = response["messages"][-1]

In [ ]:
last_msg.content

In [ ]:
user_text = "Покажи минимальную и максимальную дату продаж"

msg = {"messages": [{"role": "user", "content": user_text}]}
config = {"configurable": {"thread_id": str(1)}}
print(config)
response = graph.invoke(msg, config)
last_msg = response["messages"][-1]

In [ ]:
response

In [ ]:
[m for in response["messages"] if m.role == ""]

In [ ]:
type(response["messages"][-1])

### Tests

In [1]:
import uuid
import importlib
import sys
from httpx import request
import pytest
from typing import Dict, List, Any, Tuple
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

from langsmith import testing as t

from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.types import Command
import os
from dotenv import load_dotenv

load_dotenv(".env", override=True)


sys.path.append("/Users/aziz/Documents/repos/shai-hackathon")
import tests.evaluation_obs_sql
import app.agent as agent
import app.prompts as prompts


graph = agent.build_agent_with_router()

/Users/aziz/Documents/repos/shai-hackathon/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RESPONSE_CRITERIA_SYSTEM_PROMPT = """
You are an evaluator of SQL generation for a natural language to SQL agent.

I will give you:
1. The user request.
2. A list of calls made by the agent to tools (the candidate SQL).
3. The golden (reference) SQL query.

Your task:
- Evaluate two metrics:
  1. **Exec accuracy**: Does the candidate query produce the same result as the golden query, even if the syntax or formatting differs? (Yes/No)
  2. **Exact match**: Is the candidate query logically equivalent to the golden query? 
     - Ignore whitespace and capitalization.
     - Ignore differences in column aliases (e.g. `AS col_name`).
     - Ignore ordering of selected columns if the semantics are the same.
     - Focus only on whether the same tables, filters, joins, and aggregations are applied.

Return STRICT JSON in this format:
{
  "exec_accuracy": "<Yes/No>",
  "exact_match": "<Yes/No>",
  "explanation": "<short explanation why>"
}

Example:

User request: "How many stores are in the database?"
List of calls: "SELECT COUNT(*) AS total_stores FROM dict_store;"
Golden SQL: "SELECT COUNT(*) FROM dict_store;"

Output:
{
  "exec_accuracy": "Yes",
  "exact_match": "Yes",
  "explanation": "Both queries count the total number of stores from the same table. The only difference is the alias, which should be ignored."
}
"""



In [3]:
import tests.evaluation_obs_sql as e_sql

In [4]:
test_cases = []
for req, criteria in zip(e_sql.sql_reqsuests, e_sql.sql_answers):
    test_cases.append((req, criteria))

In [5]:
evaluator_model = os.getenv("EVALUATOR_MODEL")
evaluator_api_base = os.getenv("EVALUATOR_API_BASE")
evaluator_api_key = os.getenv("EVALUATOR_API_KEY")


class SQLEvaluation(BaseModel):
    """Evaluation of candidate SQL against golden SQL across two criteria: execution accuracy and exact match."""

    exec_accuracy: bool = Field(
        description="Does the candidate SQL produce the same result as the golden SQL (ignoring syntax/formatting differences)?"
    )
    exec_accuracy_justification: str = Field(
        description="Explain why the candidate SQL does or does not produce the same result."
    )

    exact_match: bool = Field(
        description="Is the candidate SQL textually identical to the golden SQL (ignoring whitespace and capitalization)?"
    )
    exact_match_justification: str = Field(
        description="Explain why the candidate SQL matches or differs from the golden SQL text."
    )


criteria_eval_llm = init_chat_model(
    evaluator_model,
    openai_api_base=evaluator_api_base,
    openai_api_key=evaluator_api_key,
)
criteria_eval_structured_llm = criteria_eval_llm.with_structured_output(SQLEvaluation)

In [6]:
def format_messages_string(messages: List[Any]) -> str:
    """Format messages into a single string for analysis."""
    return "\n".join(message.pretty_repr() for message in messages)

In [7]:
eval_results = []

for test_case in test_cases[8:]:
    req = test_case[0]
    golden_sql = test_case[1]

    msg = {"messages": [{"role": "user", "content": req}]}
    config = {"configurable": {"thread_id": str(req)}}

    result = graph.invoke(msg, config)
    all_messages_str = format_messages_string(result["messages"])

    eval_result = criteria_eval_structured_llm.invoke(
        [
            {"role": "system", "content": RESPONSE_CRITERIA_SYSTEM_PROMPT},
            {
                "role": "user",
                "content": f"""\n\n Request: {req}
                            Tool calls: \n\n {all_messages_str} 
                            \n\n Golden SQL: {golden_sql} \n\n 
                Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation.""",
            },
        ]
    )
    eval_results.append(eval_result)

 - TOOL CALL: list_tables()
 - TOOL CALL: describe_table(dict_store)
 - TOOL CALL: describe_table(fact_sales)
 - TOOL CALL: execute_query(SELECT ds.store_name, SUM(fs.sales_tg) as total_revenue FROM fact_sales fs JOIN dict_store ds ON fs.store_id = ds.store_id GROUP BY ds.store_name ORDER BY total_revenue DESC)
 - TOOL CALL: list_tables()
 - TOOL CALL: describe_table(fact_cost)
 - TOOL CALL: execute_query(SELECT store_id, sku_id, cost FROM fact_cost)


In [8]:
[e.exact_match for e in eval_results]

[False, True]

In [9]:

eval_results[0]

SQLEvaluation(exec_accuracy=True, exec_accuracy_justification="The candidate SQL and the golden SQL both aggregate total sales (SUM of sales_tg) per store and join the same tables on store_id. The only difference is an added 'ORDER BY total_revenue DESC' in the candidate, which only changes the row order and does not alter the actual results (store names and their corresponding total revenues). Thus, the output data is the same for both queries, meeting the user's request.", exact_match=False, exact_match_justification="The candidate SQL includes 'ORDER BY total_revenue DESC', which is not present in the golden SQL. While this doesn't affect the query result content, it is a logical difference in how results are presented, violating strict exact match per criteria.")

In [1]:
[e.exec_accuracy for e in eval_results]

NameError: name 'eval_results' is not defined

In [10]:
eval_results[0]

SQLEvaluation(exec_accuracy=False, exec_accuracy_justification='The candidate SQL counts the number of distinct store_id values in the fact_sales table, which may not match the total number of stores listed in dict_store. The golden SQL counts all entries in dict_store, which is the authoritative list of stores. If fact_sales is missing stores (e.g., stores without recorded sales), the count will differ.', exact_match=False, exact_match_justification="The candidate SQL uses a different table (fact_sales) and counts DISTINCT store_id, whereas the golden SQL uses dict_store and counts all rows. The tables and aggregation methods differ, so it's not a logical match.")

In [11]:
import app.tools as tools

In [13]:
tools.list_tables()

 - TOOL CALL: list_tables()


['dict_store', 'dict_sku', 'fact_sales', 'metadata', 'fact_bal', 'fact_cost']